In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('../data/21-25.csv')

In [9]:
from getData import get_stock_data, sp_list

def get_df(symbol):
    data = get_stock_data(symbol, '2021-01-01', '2025-01-01')
    return pd.DataFrame(data)

df_dict = {symbol: get_df(symbol) for symbol in sp_list}

In [10]:
def calculate_std_dev(df, N):
    """
    Calculate the standard deviation of daily returns over the past N days.
    """

    df['daily_return'] = df['close'].pct_change()

    std_dev = df['daily_return'].rolling(window=N).std().iloc[-1]

    return std_dev

In [11]:
def calculate_moving_average(df, indicator, N):
    """
    Calculate the N-day moving average of indicator in the dataframe.
    """
    moving_average = df[indicator].rolling(window=N).mean().iloc[-1]
    return moving_average

In [12]:
def calculate_slope_pe_movement(symbol, df):
    scaler = StandardScaler()
    sub_df = df.loc[df['symbol'] == symbol].copy()

    # scale PE ratio
    sub_df.loc[:, 'pe_scaled'] = scaler.fit_transform(sub_df['priceEarningsRatio'].values.reshape(-1, 1))

    # create a dummy x range for corresponding PE
    sub_df.loc[:, 'x_range'] = np.linspace(0, len(sub_df) - 1, len(sub_df))
    sub_df = sub_df[['x_range', 'pe_scaled']]

    # calculate slope of PE ratio movement
    slope = np.polyfit(sub_df['x_range'], sub_df['pe_scaled'].values.flatten(), 1)[0]

    return slope

In [13]:
columns_to_keep = [
    'symbol', 'calendarYear', 'period',
    'currentRatio', 'quickRatio',
    'returnOnEquity', 'returnOnAssets', 'netProfitMargin',
    'priceEarningsRatio', 'priceBookValueRatio', 'priceToSalesRatio',
    'freeCashFlowPerShare', 'operatingCashFlowPerShare', 'cashFlowToDebtRatio',
    'debtEquityRatio', 'longTermDebtToCapitalization',
    'assetTurnover', 'inventoryTurnover'
]

existing_columns = [col for col in columns_to_keep if col in df.columns]

filtered_df = df[existing_columns]

In [ ]:
def calculate_column_slope(column, sub_df):
    scaler = StandardScaler()

    # scale column
    sub_df.loc[:, 'var_scaled'] = scaler.fit_transform(sub_df[column].values.reshape(-1, 1))

    # create a dummy x range for corresponding PE
    sub_df.loc[:, 'x_range'] = np.linspace(0, len(sub_df) - 1, len(sub_df))
    sub_df = sub_df[['x_range', 'var_scaled']]

    # calculate slope of PE ratio movement
    slope = np.polyfit(sub_df['x_range'], sub_df['pe_scaled'].values.flatten(), 1)[0]

    return slope, moving_average

def ma_slope_column(sub_df, column):
    scaler = StandardScaler()
    moving_average = sub_df[column].rolling(window=len(sub_df)).mean().iloc[-1] # not sure if this is calculating in the right temporal direction

    # standardize column values
    standardized_values = scaler.fit_transform(sub_df[column].values.reshape(-1, 1)).flatten()

    # calculate slope of the line of best fit
    x_range = np.arange(len(standardized_values))
    slope = np.polyfit(x_range, standardized_values, 1)[0]

    return moving_average, slope

columns_to_analyze = [
    'currentRatio', 'quickRatio',
    'returnOnEquity', 'returnOnAssets', 'netProfitMargin',
    'priceEarningsRatio', 'priceBookValueRatio', 'priceToSalesRatio',
    'freeCashFlowPerShare', 'operatingCashFlowPerShare', 'cashFlowToDebtRatio',
    'debtEquityRatio', 'longTermDebtToCapitalization',
    'assetTurnover', 'inventoryTurnover'
]

In [15]:
data = {}

for symbol in df['symbol'].unique():
    sub_df = df.loc[df['symbol'] == symbol].copy()

    column_data = {}
    for column in columns_to_analyze:
        ma, slope = ma_slope_column(sub_df, column)
        column_data[f'{column}_ma'] = ma
        column_data[f'{column}_slope'] = slope

    volatility_90 = calculate_std_dev(sub_df, 90)

    sub_df_dict = df_dict[symbol]

    close_ma_30 = calculate_moving_average(sub_df_dict, 'close', 30)
    close_ma_90 = calculate_moving_average(sub_df_dict, 'close', 90)
    close_ma_200 = calculate_moving_average(sub_df_dict, 'close', 200)

    volume_ma_30 = calculate_moving_average(sub_df_dict, 'volume', 30)
    volume_ma_90 = calculate_moving_average(sub_df_dict, 'volume', 90)
    volume_ma_200 = calculate_moving_average(sub_df_dict, 'volume', 200)

    data[symbol] = {
        volatility_90: volatility_90,
        'close_ma_30': close_ma_30,
        'close_ma_90': close_ma_90,
        'close_ma_200': close_ma_200,
        'volume_ma_30': volume_ma_30,
        'volume_ma_90': volume_ma_90,
        'volume_ma_200': volume_ma_200,
        **column_data
    }


No longer want to calculate moving average for features. Simply take most recent instance.

In [ ]:
def get_most_recent(df, symbol):
    return df.loc[df['symbol'] == symbol].iloc[0]
    

symbol                                        LMT
reportedCurrency                              USD
cik                                      936468.0
fillingDate                            2025-01-28
acceptedDate                  2025-01-28 16:12:50
                                     ...         
priceEarningsToGrowthRatio              -0.807769
priceSalesRatio                          6.158406
dividendYield                            0.006784
enterpriseValueMultiple                 90.854944
priceFairValue                          18.108612
Name: 7577, Length: 122, dtype: object

In [16]:
finaldf = pd.DataFrame(data).T
finaldf.dropna(axis=1, how='all', inplace=True)

finaldf.ffill(inplace=True) # forward fill NA

finaldf.to_csv('finaldf.csv')